In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label', axis = 1)

In [6]:
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
from tensorflow.keras.layers import Embedding

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
from tensorflow.keras.preprocessing.text import one_hot

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [13]:
voc_size = 5000

In [14]:
message = X.copy()

In [15]:
message.reset_index(inplace = True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:


from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()



In [20]:


corpus = []
for i in range(0, len(message)):
   
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    



In [21]:
from tensorflow.keras.layers import Bidirectional

In [22]:
len(corpus)

18285

In [23]:
message['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [24]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [25]:
sent_length = 20
embeded_docs = pad_sequences(onehot_repr, padding = "pre", maxlen = sent_length)

In [26]:
embeded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  196,
       4104,  553, 2754, 1178, 4506, 4581,  999,  623, 1015], dtype=int32)

In [27]:
dims = 40
model = Sequential()

2022-03-03 18:13:20.751958: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [28]:
model.add(Embedding(voc_size, dims, input_length = sent_length))

In [29]:
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation = "sigmoid"))

In [30]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
X_final = np.array(embeded_docs)

In [32]:
y_final = np.array(y)

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [35]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

2022-03-03 18:13:22.861845: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
192/192 [==============================] - 15s 55ms/step - loss: 0.2884 - accuracy: 0.8681 - val_loss: 0.2122 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1346 - accuracy: 0.9472 - val_loss: 0.2029 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0957 - accuracy: 0.9638 - val_loss: 0.2433 - val_accuracy: 0.9150
Epoch 4/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0680 - accuracy: 0.9757 - val_loss: 0.2932 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0449 - accuracy: 0.9823 - val_loss: 0.3390 - val_accuracy: 0.9123
Epoch 6/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0335 - accuracy: 0.9882 - val_loss: 0.4227 - val_accuracy: 0.9099
Epoch 7/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0224 - accuracy: 0.9920 - val_loss: 0.4436 - val_accuracy: 0.

In [36]:
y_pred = model.predict(X_test)

In [37]:
threshold = 0.5
a = np.where(y_pred>=threshold, 1,0)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
print(accuracy_score(y_test, a))

0.907705053852527


In [40]:
df_test = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [41]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [42]:
df_test.shape

(5200, 4)

In [43]:
df_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [44]:
df_test['title'] = df_test['title'].fillna(df_test['title'].mode()[0])

In [45]:
df_test.isnull().sum()

id          0
title       0
author    503
text        7
dtype: int64

In [46]:
submit = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [47]:
messages = df_test

In [48]:
messages.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [49]:
corpus = []
for i in range(0, len(messages)):
   
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [50]:
len(corpus)

5200

In [51]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [52]:
sent_length = 20
embeded_docs = pad_sequences(onehot_repr, padding = "pre", maxlen = sent_length)

In [53]:
X_test_final = np.array(embeded_docs)

In [54]:
y_pred = model.predict(X_test_final)

In [55]:
threshold = 0.5
a = np.where(y_pred>=threshold, 1,0)

In [56]:
submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [57]:
label = pd.DataFrame(a, columns = ['label'])

In [58]:
label.head()

,label
0,0
1,1
2,0
3,0
4,1


In [59]:
submit.drop('label', inplace = True, axis = 1)

In [60]:
submit = pd.concat([submit, label], axis = 1)

In [61]:
submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [62]:
submit.to_csv('submit.csv', index = False)